# Pathology prediction (fracture)

@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the fracture pathology using the CSV embeddings file

## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples :  0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value. 

The model is based on binary classification for each target chest pathology.

Final sample size for the fracture pathology is : (N = 557)

#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.FRACTURE, 
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }
# Fixed parameters to pass
fixed_params = {
    'seed': 42,
    'eval_metric': 'logloss',
    'verbosity': 0
}

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Fracture", experiment_name="CP_Fracture", filepath="./results/fracture")

evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='kfold',
    outer_fold=5,
    outer_strategy='stratifiedkfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps,  # Grid d'hyperparamètres
    fixed_params=fixed_params # Paramètres fixes
)

2024-10-20 17:14:14,195	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=189451) Outer fold 1
(run_fold pid=189455) Train indices: [  0   1   2   3   4   6   8   9  10  11  12  13  14  15  17  18  19  20
(run_fold pid=189455)   23  24  25  26  27  28  29  32  33  34  36  37  38  39  40  42  43  44
(run_fold pid=189455)   45  46  47  48  49  50  51  53  54  55  56  57  59  60  61  62  65  66
(run_fold pid=189455)   67  68  69  70  71  72  73  74  75  76  77  78  80  81  82  83  85  86
(run_fold pid=189455)   87  88  89  90  91  92  93  94  96  97  98  99 100 102 104 105 106 107
(run_fold pid=189455)  108 109 110 111 114 115 116 117 119 120 122 123 125 126 127 128 129 130
(run_fold pid=189455)  131 132 133 135 136 137 138 139 141 142 143 144 145 147 150 151 153 154
(run_fold pid=189455)  155 156 158 159 160 161 162 163 164 166 167 168 169 170 172 173 174 175
(run_fold pid=189455)  176 178 179 181 182 183 184 185 186 187 188 189 190 191 193 194 195 197
(run_fold pid=189455)  198 199 200 201 202 203 204 205 206 207 208 210 211 212 216 218 219 221


Processing:   0%|          | 0/4 [00:00<?, ?it/s]


(run_fold pid=189456) Configuring PyCaret for outer fold 3
(run_fold pid=189437) Outer fold 2 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(run_fold pid=189451) Train indices: [  0   3   5   6   7   8   9  10  11  12  13  15  16  18  19  20  21  22 [repeated 4x across cluster]
(run_fold pid=189451) Test indices: [  1   2   4  14  17  23  28  29  36  38  43  45  50  55  60  61  62  67 [repeated 4x across cluster]


Processing:  75%|███████▌  | 3/4 [00:23<00:08,  8.47s/it] [repeated 5x across cluster]


(run_fold pid=189447)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=189447) Fold                                                          
(run_fold pid=189447) 0       0.9722  0.8744     1.0  0.9718  0.9857  0.4894  0.5692
(run_fold pid=189447) 1       0.9437  0.8909     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=189447) 2       0.9296  0.7212     1.0  0.9296  0.9635  0.0000  0.0000
(run_fold pid=189447) 3       0.9577  0.7091     1.0  0.9565  0.9778  0.5535  0.6186
(run_fold pid=189447) 4       0.9437  0.7424     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=189447) Mean    0.9494  0.7876     1.0  0.9487  0.9736  0.3355  0.4112
(run_fold pid=189447) Std     0.0145  0.0785     0.0  0.0144  0.0075  0.1920  0.2182
(run_fold pid=189455) Configuring PyCaret for outer fold 4 [repeated 4x across cluster]


(run_fold pid=189456) 0       0.9583  0.7391     1.0  0.9583  0.9787  0.0000  0.0000
(run_fold pid=189456) 1       0.9577  0.6667     1.0  0.9565  0.9778  0.5535  0.6186
(run_fold pid=189456) 2       0.9437  0.7909     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=189456) 3       0.9437  0.6424     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=189456) 4       0.9577  0.7788     1.0  0.9565  0.9778  0.5535  0.6186
(run_fold pid=189447) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=189455) 0       0.9722  0.8164     1.0  0.9718  0.9857  0.4894  0.5692
(run_fold pid=189455) 1       0.9437  0.7333     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=189455) 2       0.9577  0.8030     1.0  0.9565  0.9778  0.5535  0.6186
(run_fold pid=189455) 3       0.9296  0.8000     1.0  0.9296  0.9635  0.0000  0.0000
(run_fold pid=189455) 4       0.9437  0.9818     1.0  0.9429  0.9706  0.3173  0.4342
(run_fold pid=189437) 0       0.9583  0.8599     1.0

Processing:  75%|███████▌  | 3/4 [00:24<00:08,  8.84s/it] [repeated 4x across cluster]


(run_fold pid=189456) Transformation Pipeline and Model Successfully Saved
(run_fold pid=189451)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC [repeated 4x across cluster]
(run_fold pid=189451) Fold                                                           [repeated 4x across cluster]
(run_fold pid=189451) Mean    0.9522  0.7734     1.0  0.9514  0.9751  0.3989  0.4981 [repeated 4x across cluster]
(run_fold pid=189451) Std     0.0114  0.1347     0.0  0.0115  0.0060  0.1020  0.0797 [repeated 4x across cluster]
(run_fold pid=189451) Tuning hyperparameters for model xgboost with custom grid using grid search [repeated 4x across cluster]
(run_fold pid=189456)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=189456) 0  Extreme Gradient Boosting     0.955  0.7788  ...  0.9765  0.4284  0.5221
(run_fold pid=189456) 
(run_fold pid=189456) [1 rows x 8 columns]
(run_fold pid=189455) Transformation Pipeline and Model Successfully Saved
(run_

                                                          [repeated 4x across cluster]
